# Práctica 2: Transformaciones geométricas
Manuel Alejandro Torrealba Torrealba, Alejandro Alemán Alemán

### (Obligatorio – 5 puntos)
#### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.

- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.

In [4]:
import cv2 as cv
import numpy as np

# ------------------------------
# ------------------------------
# CARGA DE IMAGEN
# ------------------------------
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

# Calculamos un borde seguro (por ejemplo, la mitad del ancho/alto original a cada lado)
# Esto hace que el lienzo sea el doble de grande, dando espacio para rotar sin cortes.
top = img.shape[0] // 2
bottom = top
left = img.shape[1] // 2
right = left

# Creamos una nueva imagen con bordes negros añadidos
img = cv.copyMakeBorder(img, top, bottom, left, right, cv.BORDER_CONSTANT, value=[0, 0, 0])

# Se extraen las dimensiones de la NUEVA imagen (que ahora es más grande)
h, w = img.shape[:2]
size = (w, h)  # El tamaño destino ahora incluye el margen

def nothing(x):
    # Función vacía requerida por las trackbars (OpenCV exige una función callback)
    pass

# ------------------------------
# CREACIÓN DE VENTANA Y TRACKBARS
# ------------------------------
cv.namedWindow("Transformaciones")  # Ventana principal donde se mostrarán los resultados

# --- Traslación ---
# Las trackbars controlan el desplazamiento en X e Y.
# El rango va de 0 a 2*w o 2*h, y se resta el valor original después
# para permitir desplazamientos negativos y positivos.
cv.createTrackbar('tx', 'Transformaciones', w, 2 * w, nothing)
cv.createTrackbar('ty', 'Transformaciones', h, 2 * h, nothing)

# --- Rotación ---
# Ángulo de rotación (0–360) y posición del centro de rotación (píxeles)
cv.createTrackbar('Angulo', 'Transformaciones', 0, 360, nothing)
cv.createTrackbar('Centro X', 'Transformaciones', w // 2, w, nothing)
cv.createTrackbar('Centro Y', 'Transformaciones', h // 2, h, nothing)

# --- Escalado ---
# Factores de escala independientes para X e Y (en %)
# “Uniforme” es un interruptor (0/1) que fuerza el mismo factor en ambos ejes.
cv.createTrackbar('Uniforme', 'Transformaciones', 0, 1, nothing)
cv.createTrackbar('Escalado X', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado X', 'Transformaciones', 10)  # evitar escala cero
cv.createTrackbar('Escalado Y', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado Y', 'Transformaciones', 10)

# ------------------------------
# VALORES INICIALES (para reiniciar)
# ------------------------------
# Diccionario con los valores predeterminados de cada control.
# Se usa en la función reset_trackbars().
initial_values = {
    'tx': w,
    'ty': h,
    'Angulo': 0,
    'Centro X': w // 2,
    'Centro Y': h // 2,
    'Uniforme': 0,
    'Escalado X': 100,
    'Escalado Y': 100
}

def reset_trackbars():
    """Restablece todos los valores de las trackbars al estado inicial."""
    # Recorre el diccionario y coloca cada trackbar en su posición inicial.
    for name, val in initial_values.items():
        cv.setTrackbarPos(name, 'Transformaciones', val)
    print("Parámetros reiniciados")

# ------------------------------
# BUCLE PRINCIPAL
# ------------------------------
# Este bucle se ejecuta indefinidamente, leyendo los valores de las trackbars
# y aplicando las transformaciones correspondientes en tiempo real.
while True:
    # ----- Lectura de trackbars -----
    # Se obtienen los valores actuales de las barras.
    # En traslación se resta el ancho/alto para permitir desplazamientos negativos.
    tx = cv.getTrackbarPos('tx', 'Transformaciones') - w
    ty = cv.getTrackbarPos('ty', 'Transformaciones') - h
    ang = cv.getTrackbarPos('Angulo', 'Transformaciones')
    cx = cv.getTrackbarPos('Centro X', 'Transformaciones')
    cy = cv.getTrackbarPos('Centro Y', 'Transformaciones')
    sx = cv.getTrackbarPos('Escalado X', 'Transformaciones') / 100
    sy = cv.getTrackbarPos('Escalado Y', 'Transformaciones') / 100
    uniforme = cv.getTrackbarPos('Uniforme', 'Transformaciones')

    # Si el escalado uniforme está activado, iguala sx y sy
    if uniforme == 1:
        sx = sy

    # ------------------------------
    # COMBINACIÓN DE TRANSFORMACIONES
    # ------------------------------

    # 1️⃣ ROTACIÓN
    # Se obtiene la matriz afín de rotación 2x3 con respecto al centro (cx, cy)
    # El tercer parámetro (1.0) indica que no hay escalado adicional en la rotación.
    R = cv.getRotationMatrix2D((cx, cy), ang, 1.0)
    rotated = cv.warpAffine(img, R, size)
    # cv.warpAffine aplica la transformación afín especificada por la matriz R a toda la imagen.

    # 2️⃣ ESCALADO
    # Se crea una matriz afín manual con los factores de escala definidos.
    # No se añade traslación (última columna en ceros).
    S = np.float32([[sx, 0, 0],
                    [0, sy, 0]])
    scaled = cv.warpAffine(rotated, S, size)
    # Este paso agranda o reduce la imagen ya rotada.

    # 3️⃣ TRASLACIÓN
    # Matriz afín de desplazamiento. tx y ty indican los píxeles de movimiento.
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    final = cv.warpAffine(scaled, T, size)
    # Se aplica finalmente el desplazamiento sobre la imagen escalada.

    # Mostrar el resultado acumulado (rotada + escalada + trasladada)
    cv.imshow("Transformaciones", final)

    # ------------------------------
    # Captura de teclas
    # ------------------------------
    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC -> salir del programa
        break
    elif key == ord('r') or key == ord('R'):
        # Reinicia las trackbars al estado original
        reset_trackbars()

# Cierre seguro de ventanas (evita bloqueos en algunos entornos)
cv.destroyAllWindows()
cv.waitKey(1)

Parámetros reiniciados


-1

#### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [7]:
import cv2 as cv
import numpy as np
import sys

# ------------------------------
# Cargar imagen original
# ------------------------------
# Se carga la imagen que se va a proyectar. Si no se encuentra, se aborta la ejecución.
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    sys.exit(1)

# Dimensiones de la imagen original
h, w = img.shape[:2]

# Se crea una segunda imagen (blanca) del mismo tamaño que actuará como ventana destino
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # segunda ventana en blanco

# Copias de trabajo para dibujar los puntos de selección sin modificar las originales
clone1 = img.copy()
clone2 = img2.copy()

# Listas para almacenar las coordenadas de los puntos seleccionados en cada imagen
pts_src = []  # puntos en la imagen fuente (original)
pts_dst = []  # puntos en la imagen destino (proyección)


# ------------------------------
# Callback del ratón
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    """
    Función que se ejecuta cada vez que ocurre un evento del ratón en las ventanas.
    Permite registrar los puntos seleccionados en las imágenes original y destino.
    """
    global pts_src, pts_dst, clone1, clone2

    if event == cv.EVENT_LBUTTONDOWN:  # Detecta clic izquierdo
        # Si el parámetro 'param' es 1, el evento corresponde a la imagen fuente
        if param == 1 and len(pts_src) < 4:
            # Añadir el punto y dibujarlo en la copia de la imagen original
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)  # punto rojo
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv.imshow('Original', clone1)

        # Si 'param' es 2, el clic se hizo en la imagen destino
        elif param == 2 and len(pts_dst) < 4:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)  # punto azul
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv.imshow('Destino', clone2)

        # Cuando ya se han seleccionado 4 puntos en ambas imágenes, se ejecuta la proyección
        if len(pts_src) == 4 and len(pts_dst) == 4:
            proyectar_imagen()


# ------------------------------
# Proyección
# ------------------------------
def proyectar_imagen():
    """
    Calcula la homografía (transformación proyectiva) que mapea los cuatro puntos
    de la imagen original a los cuatro puntos seleccionados en la imagen destino.
    """
    global pts_src, pts_dst, img, img2

    # cv.getPerspectiveTransform genera una matriz 3x3 que representa la homografía.
    # Esta matriz permite transformar cualquier punto del plano original al plano destino.
    M = cv.getPerspectiveTransform(np.float32(pts_src), np.float32(pts_dst))

    # cv.warpPerspective aplica la homografía a la imagen original para obtener la proyección.
    proyectada = cv.warpPerspective(img, M, (img2.shape[1], img2.shape[0]))

    # Se muestra la imagen proyectada resultante
    cv.imshow('Proyeccion', proyectada)
    print("Transformación completada.")


# ------------------------------
# Reset
# ------------------------------
def reset():
    """
    Reinicia la selección de puntos y restaura las imágenes originales.
    Permite repetir el proceso sin cerrar el programa.
    """
    global pts_src, pts_dst, clone1, clone2
    pts_src = []
    pts_dst = []
    clone1 = img.copy()
    clone2 = img2.copy()
    cv.imshow('Original', clone1)
    cv.imshow('Destino', clone2)
    print("Se ha reseteado la selección de puntos.")


# ------------------------------
# Ventanas
# ------------------------------
# Se crean las ventanas donde se mostrarán las imágenes y se vinculan los callbacks del ratón.
cv.namedWindow('Original')
cv.namedWindow('Destino')

# El tercer parámetro (1 o 2) indica qué ventana se está usando dentro del callback.
cv.setMouseCallback('Original', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

# Instrucciones para el usuario (mostradas por consola)
print("Haz clic en 4 puntos en la imagen ORIGINAL (rojo).")
print("Luego haz clic en 4 puntos en la imagen DESTINO (azul).")
print("Presiona 'r' para resetear, o 'ESC' para salir.")

# Se muestran las dos imágenes iniciales
cv.imshow('Original', img)
cv.imshow('Destino', img2)

# ------------------------------
# Bucle principal
# ------------------------------
# Permite mantener las ventanas abiertas hasta que el usuario pulse ESC o 'r'.
while True:
    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC -> salir
        break
    elif key == ord('r'):  # R -> resetear selección de puntos
        reset()

# ------------------------------
# Cierre de ventanas
# ------------------------------
cv.destroyAllWindows()
cv.waitKey(1)


Haz clic en 4 puntos en la imagen ORIGINAL (rojo).
Luego haz clic en 4 puntos en la imagen DESTINO (azul).
Presiona 'r' para resetear, o 'ESC' para salir.
Transformación completada.
Se ha reseteado la selección de puntos.
Se ha reseteado la selección de puntos.
Se ha reseteado la selección de puntos.
Transformación completada.
Se ha reseteado la selección de puntos.
Transformación completada.
Se ha reseteado la selección de puntos.
Se ha reseteado la selección de puntos.
Transformación completada.
Se ha reseteado la selección de puntos.
Transformación completada.
Se ha reseteado la selección de puntos.


-1

#### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [8]:
import cv2
import numpy as np

# --- CARGA DE IMAGEN ---
# Se carga la imagen sobre la que se aplicará la distorsión de lente.
# Si no se encuentra, el programa se detiene.
img = cv2.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

# Se obtienen las dimensiones de la imagen
h, w = img.shape[:2]
size = (w, h)

def nothing(x):
    # Función vacía necesaria para la creación de trackbars (callback obligatorio)
    pass

# --- FUNCIÓN DE DISTORSIÓN ---
def apply_distortion(image, k1, k2):
    """
    Aplica una distorsión óptica simulada a la imagen en función
    de los coeficientes radiales k1 y k2.

    - k1 controla la distorsión de primer orden.
    - k2 controla la distorsión de segundo orden.
    """
    h, w = image.shape[:2]

    # Vector de coeficientes de distorsión (k1, k2, p1, p2)
    # Solo se usan los dos primeros (radiales). p1 y p2 son tangenciales y se dejan en 0.
    distCoeff = np.zeros((4, 1), np.float64)
    distCoeff[0, 0] = k1
    distCoeff[1, 0] = k2

    # Matriz intrínseca de cámara (3x3)
    # Define las propiedades ópticas de la cámara simulada (foco y centro óptico).
    cam = np.eye(3, dtype=np.float32)
    cam[0, 2] = w / 2.0   # coordenada x del centro de la imagen
    cam[1, 2] = h / 2.0   # coordenada y del centro de la imagen
    cam[0, 0] = 10.0      # longitud focal en píxeles (f_x)
    cam[1, 1] = 10.0      # longitud focal en píxeles (f_y)

    # cv2.undistort() corrige la distorsión de una cámara, pero aquí se usa
    # de forma inversa: se modifica la matriz para simular un efecto de distorsión.
    distorted_img = cv2.undistort(image, cam, distCoeff)
    return distorted_img

# --- CREAR VENTANA Y TRACKBARS ---
cv2.namedWindow("Distorsion")

# Cada trackbar controla uno de los coeficientes (k1 y k2)
# El rango 0–1000 se interpreta como -0.005 a +0.005 tras la conversión,
# permitiendo distorsión de barril (k<0) o de cojín (k>0).
cv2.createTrackbar('K1', 'Distorsion', 500, 1000, nothing)
cv2.createTrackbar('K2', 'Distorsion', 500, 1000, nothing)

# --- VALORES INICIALES ---
# Valor 500 = punto neutro (equivale a 0 tras la conversión)
initial_values = {'K1': 500, 'K2': 500}

def reset_trackbars():
    """
    Restablece las barras de desplazamiento a sus valores iniciales.
    """
    for name, val in initial_values.items():
        cv2.setTrackbarPos(name, 'Distorsion', val)
    print("Parámetros reiniciados")

# --- BUCLE PRINCIPAL ---
while True:
    # Se leen los valores actuales de las trackbars.
    # Se convierten a rango (-0.005, +0.005) dividiendo entre 100000.
    k1 = (cv2.getTrackbarPos('K1', 'Distorsion') - 500) / 100000
    k2 = (cv2.getTrackbarPos('K2', 'Distorsion') - 500) / 100000

    # Aplicar distorsión a la imagen según los parámetros actuales
    distorsion_img = apply_distortion(img, k1, k2)

    # Mostrar los valores actuales en la imagen como texto superpuesto
    texto = f"K1={k1:.5f}, K2={k2:.5f} (R para reset)"
    cv2.putText(distorsion_img, texto, (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Mostrar la imagen distorsionada en tiempo real
    cv2.imshow("Distorsion", distorsion_img)

    # Control de teclado
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC → salir
        break
    elif key == ord('r') or key == ord('R'):
        reset_trackbars()

# Cierre robusto de ventanas
cv2.destroyAllWindows()
cv2.waitKey(1)


Parámetros reiniciados
Parámetros reiniciados


-1

### (Optativo – 5 puntos)

- Marcar el punto de giro con el ratón.
- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.
- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en
una imagen destino y realizar la transformación afín.
- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen.
- Otras aportaciones

#### Marcar el punto de giro con el ratón.

In [9]:
# Marcar el punto de giro con el ratón.

import cv2 as cv
import numpy as np

# --- CARGA DE IMAGEN ---
# Se carga la imagen sobre la que se aplicarán las rotaciones interactivas.
img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])  # tamaño de salida (ancho, alto)
h, w = img.shape[:2]
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    # Función vacía necesaria para crear la trackbar (callback requerido por OpenCV)
    pass

# --- INICIALIZACIÓN DEL CENTRO DE ROTACIÓN ---
# Por defecto, el punto de giro se sitúa en el centro de la imagen.
cx = w // 2
cy = h // 2

# --- CALLBACK DEL RATÓN ---
def point_turn(event, x, y, flags, param):
    """
    Permite actualizar el centro de rotación haciendo clic en la imagen.
    Cada clic izquierdo establece nuevas coordenadas (cx, cy).
    """
    global cx, cy
    
    if event == cv.EVENT_LBUTTONDOWN:  # clic izquierdo
        cx, cy = x, y
        print(f"Nuevo punto de rotación: ({cx}, {cy})")  # muestra en consola

# --- CONFIGURACIÓN DE LA VENTANA Y CONTROLES ---
cv.namedWindow("Rotacion")

# Asocia la función 'point_turn' a los eventos del ratón en la ventana.
cv.setMouseCallback("Rotacion", point_turn)

# Trackbar para controlar el ángulo de rotación (0–360 grados)
cv.createTrackbar('Angulo', 'Rotacion', 0, 360, nothing)

# --- BUCLE PRINCIPAL ---
while True:
    # Se obtiene el valor actual del ángulo desde la trackbar
    ang = cv.getTrackbarPos('Angulo', 'Rotacion')
    
    # Calcular la matriz afín de rotación en torno al punto (cx, cy)
    # cv.getRotationMatrix2D genera una matriz 2x3 que combina rotación + traslación
    # para mantener el centro de rotación en las coordenadas seleccionadas.
    M = cv.getRotationMatrix2D((cx, cy), ang, 1.0)
    
    # Aplicar la transformación afín (rotación)
    imageR = cv.warpAffine(img, M, size)

    # --- REPRESENTACIÓN VISUAL ---
    # Dibuja un pequeño marcador en el punto de rotación:
    # círculo interior amarillo + borde rojo para destacar la posición exacta.
    cv.circle(imageR, (cx, cy), 5, (0, 255, 255), -1)
    cv.circle(imageR, (cx, cy), 8, (0, 0, 255), 2)
    
    # Superponer texto informativo con el ángulo y coordenadas del centro
    info_text = f"Angulo: {ang} | Centro: ({cx}, {cy})"
    cv.putText(imageR, info_text, (10, 30), 
                cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Mostrar la imagen actualizada
    cv.imshow('Rotacion', imageR)

    # --- CONTROL DE TECLAS ---
    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC -> salir del programa
        break
    elif key == ord('r') or key == ord('R'):
        # Reinicia el punto de rotación al centro de la imagen
        cx = w // 2
        cy = h // 2
        print("Punto de rotación reiniciado al centro")

# --- CIERRE ROBUSTO DE VENTANAS ---
cv.destroyAllWindows()
cv.waitKey(1)


Nuevo punto de rotación: (223, 186)
Nuevo punto de rotación: (222, 169)
Nuevo punto de rotación: (102, 65)
Punto de rotación reiniciado al centro
Punto de rotación reiniciado al centro


-1

#### Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.

In [10]:
# Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.
import cv2 as cv
import numpy as np

# --- CARGA DE IMAGEN ---
img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])  # tamaño de salida de warpAffine (ancho, alto)
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    # Función dummy necesaria para posibles trackbars
    pass

h, w = img.shape[:2]

# ------------------------------
# VARIABLES DE ESTADO
# ------------------------------
draw = False   # Indica si el usuario está "arrastrando" con el ratón
ix = 0         # Offset inicial en X respecto al clic
iy = 0         # Offset inicial en Y respecto al clic
tx = 0         # Traslación acumulada en X
ty = 0         # Traslación acumulada en Y

# ------------------------------
# CALLBACK DEL RATÓN
# ------------------------------
def move_image(event, x, y, flags, param):
    """
    Permite arrastrar la imagen en tiempo real mediante el ratón.
    Se guardan offsets relativos para evitar "saltos" al iniciar el arrastre.
    """
    global draw, ix, iy, tx, ty, img
    
    if event == cv.EVENT_LBUTTONDOWN:
        # Activamos modo arrastre
        draw = True
        # Guardamos el desplazamiento inicial entre el clic y la traslación actual.
        # Esto evita que la imagen "pegue un salto" cuando se empieza a arrastrar.
        ix = x - tx
        iy = y - ty
        
    elif event == cv.EVENT_MOUSEMOVE:
        if draw:
            # Mientras se arrastra, actualizamos tx y ty según la posición del ratón.
            # De esta forma, la imagen sigue al cursor de manera fluida.
            tx = x - ix
            ty = y - iy
        
    elif event == cv.EVENT_LBUTTONUP:
        # Al soltar el botón, se desactiva el arrastre.
        draw = False

# ------------------------------
# CONFIGURACIÓN DE LA VENTANA
# ------------------------------
cv.namedWindow("Traslacion con el raton")
cv.setMouseCallback("Traslacion con el raton", move_image)

# ------------------------------
# BUCLE PRINCIPAL
# ------------------------------
while True:
    # Matriz de traslación afín (2x3)
    # [1 0 tx]
    # [0 1 ty]
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])

    # Aplicar la traslación a la imagen
    imageT = cv.warpAffine(img, T, (w, h))
    
    # Mostrar la imagen desplazada
    cv.imshow('Traslacion con el raton', imageT)

    # Gestión del teclado
    if cv.waitKey(1) & 0xFF == 27:  # ESC → salir
        break
    elif cv.waitKey(1) & 0xFF == ord('r') or cv.waitKey(1) & 0xFF == ord('R'):
        # Reinicia la traslación
        tx = 0
        ty = 0

# Cierre seguro de ventanas
cv.destroyAllWindows()
cv.waitKey(1)


-1

#### Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.

In [11]:
import cv2 as cv
import numpy as np

# ------------------------------------------
# CARGA DE VIDEO
# ------------------------------------------
ruta_video = "videos/times-square.mp4"
cap = cv.VideoCapture(ruta_video)

if not cap.isOpened():
    print("Error: no se pudo abrir el vídeo")
    exit()

# Leemos un frame inicial para obtener tamaño
ret, frame = cap.read()
if not ret:
    print("Error: no se pudo leer el primer frame")
    exit()

# ----------- PADDING PARA EVITAR RECORTES -----------
top = frame.shape[0] // 2
bottom = top
left = frame.shape[1] // 2
right = left

frame = cv.copyMakeBorder(frame, top, bottom, left, right, cv.BORDER_CONSTANT, value=[0,0,0])

h, w = frame.shape[:2]
size = (w, h)

def nothing(x):
    pass


# ------------------------------------------
# CREACIÓN DE VENTANA + TRACKBARS
# ------------------------------------------
cv.namedWindow("Transformaciones")

# Traslación
cv.createTrackbar('tx', 'Transformaciones', w, 2*w, nothing)
cv.createTrackbar('ty', 'Transformaciones', h, 2*h, nothing)

# Rotación
cv.createTrackbar('Angulo', 'Transformaciones', 0, 360, nothing)
cv.createTrackbar('Centro X', 'Transformaciones', w//2, w, nothing)
cv.createTrackbar('Centro Y', 'Transformaciones', h//2, h, nothing)

# Escalado
cv.createTrackbar('Uniforme', 'Transformaciones', 0, 1, nothing)
cv.createTrackbar('Escalado X', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado X', 'Transformaciones', 10)
cv.createTrackbar('Escalado Y', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado Y', 'Transformaciones', 10)

# Valores iniciales
initial_values = {
    'tx': w,
    'ty': h,
    'Angulo': 0,
    'Centro X': w//2,
    'Centro Y': h//2,
    'Uniforme': 0,
    'Escalado X': 100,
    'Escalado Y': 100
}

def reset_trackbars():
    for name, val in initial_values.items():
        cv.setTrackbarPos(name, 'Transformaciones', val)
    print("Parámetros reiniciados")


# ------------------------------------------
# BUCLE PRINCIPAL
# ------------------------------------------
while True:

    # Leer siguiente frame
    ret, frame = cap.read()
    if not ret:
        print("Fin del vídeo")
        break

    # Aplicar padding en cada frame
    frame = cv.copyMakeBorder(frame, top, bottom, left, right,
                              cv.BORDER_CONSTANT, value=[0,0,0])

    # Leer valores
    tx = cv.getTrackbarPos('tx', 'Transformaciones') - w
    ty = cv.getTrackbarPos('ty', 'Transformaciones') - h
    ang = cv.getTrackbarPos('Angulo', 'Transformaciones')
    cx = cv.getTrackbarPos('Centro X', 'Transformaciones')
    cy = cv.getTrackbarPos('Centro Y', 'Transformaciones')
    sx = cv.getTrackbarPos('Escalado X', 'Transformaciones') / 100
    sy = cv.getTrackbarPos('Escalado Y', 'Transformaciones') / 100
    uniforme = cv.getTrackbarPos('Uniforme', 'Transformaciones')

    if uniforme == 1:
        sx = sy

    # -----------------------------
    # ROTACIÓN
    R = cv.getRotationMatrix2D((cx, cy), ang, 1.0)
    rotated = cv.warpAffine(frame, R, size)

    # -----------------------------
    # ESCALADO
    S = np.float32([[sx, 0, 0],
                    [0, sy, 0]])
    scaled = cv.warpAffine(rotated, S, size)

    # -----------------------------
    # TRASLACIÓN
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    final = cv.warpAffine(scaled, T, size)

    # -----------------------------
    # Mostrar
    cv.imshow("Transformaciones", final)

    # Teclas
    key = cv.waitKey(10) & 0xFF
    if key == 27:   # ESC
        break
    elif key in (ord('r'), ord('R')):
        reset_trackbars()


# ------------------------------------------
# CIERRE ROBUSTO DE VENTANAS
# ------------------------------------------
cap.release()
cv.destroyAllWindows()
cv.waitKey(1)


Parámetros reiniciados


-1

In [12]:
import cv2 as cv
import numpy as np

# ------------------------------
# Cargar el vídeo
# ------------------------------
cap = cv.VideoCapture('videos/times-square.mp4')
if not cap.isOpened():
    print("Error: no se pudo cargar el vídeo")
    exit()

# Leer primer fotograma
ret, frame = cap.read()
if not ret:
    print("Error: no se pudo leer el primer fotograma del vídeo")
    cap.release()
    exit()

h, w = frame.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # ventana destino blanca

# Copias iniciales
clone1 = frame.copy()
clone2 = img2.copy()

# Listas de puntos
pts_src = []
pts_dst = []

M = None  # matriz de homografía

# ------------------------------
# Función de reset
# ------------------------------
def reset():
    """Reinicia la selección de puntos y las ventanas."""
    global pts_src, pts_dst, clone1, clone2, M

    pts_src.clear()
    pts_dst.clear()
    M = None
    clone1 = frame.copy()
    clone2 = img2.copy()

    cv.imshow('Fuente (Video)', clone1)
    cv.imshow('Destino', clone2)
    print("🔄 Reinicio completado: selecciona nuevamente los puntos.")

# ------------------------------
# Callback del ratón
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    global pts_src, pts_dst, clone1, clone2, M

    if event == cv.EVENT_LBUTTONDOWN:
        if param == 1 and len(pts_src) < 4:
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv.imshow('Fuente (Video)', clone1)

        elif param == 2 and len(pts_dst) < 4:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv.imshow('Destino', clone2)

        # Cuando se seleccionan 4 puntos en ambas imágenes
        if len(pts_src) == 4 and len(pts_dst) == 4:
            M = cv.getPerspectiveTransform(np.float32(pts_src), np.float32(pts_dst))
            print("✅ Homografía calculada. Reproduciendo vídeo proyectado...")
            cv.destroyWindow('Fuente (Video)')
            cv.destroyWindow('Destino')
            reproducir_video()

# ------------------------------
# Reproducir el vídeo proyectado
# ------------------------------
def reproducir_video():
    global cap, M, h, w

    cap.set(cv.CAP_PROP_POS_FRAMES, 0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        proyectado = cv.warpPerspective(frame, M, (w, h))
        cv.imshow('Proyeccion del Video', proyectado)

        key = cv.waitKey(30) & 0xFF
        if key == 27:  # ESC
            break

    cv.destroyWindow('Proyeccion del Video')

# ------------------------------
# Interfaz inicial
# ------------------------------
cv.namedWindow('Fuente (Video)')
cv.namedWindow('Destino')

cv.setMouseCallback('Fuente (Video)', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("🖱️ Haz clic en 4 puntos en el primer fotograma del VÍDEO (rojo).")
print("🖱️ Luego haz clic en 4 puntos en la imagen DESTINO (azul).")
print("💡 Pulsa 'r' para reiniciar la selección o 'ESC' para salir.")

cv.imshow('Fuente (Video)', clone1)
cv.imshow('Destino', clone2)

# ------------------------------
# Bucle de control principal
# ------------------------------
# ------------------------------
# Bucle de control principal
# ------------------------------
while True:
    key = cv.waitKey(1) & 0xFF

    try:
        # Si las ventanas ya se cerraron manualmente → salir
        if cv.getWindowProperty('Fuente (Video)', cv.WND_PROP_VISIBLE) < 1 or \
           cv.getWindowProperty('Destino', cv.WND_PROP_VISIBLE) < 1:
            break
    except cv.error:
        # Si alguna ventana ya no existe (macOS especialmente)
        break

    if key == 27:  # ESC
        break
    elif key == ord('r') or key == ord('R'):
        reset()

# ------------------------------
# Liberar recursos
# ------------------------------
cap.release()
cv.destroyAllWindows()
cv.waitKey(1)

#TODO: el R reset no funciona bien


🖱️ Haz clic en 4 puntos en el primer fotograma del VÍDEO (rojo).
🖱️ Luego haz clic en 4 puntos en la imagen DESTINO (azul).
💡 Pulsa 'r' para reiniciar la selección o 'ESC' para salir.
✅ Homografía calculada. Reproduciendo vídeo proyectado...


-1

In [14]:
import cv2
import numpy as np

video = cv2.VideoCapture('videos/times-square.mp4')
if not video.isOpened():
    print("Error: no se pudo abrir el vídeo")
    exit()

ret, frame = video.read()
if not ret:
    print("Error: vídeo vacío")
    exit()

def nothing(x): pass

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0], distCoeff[1,0] = k1, k2
    cam = np.eye(3, dtype=np.float32)
    cam[0,2], cam[1,2] = w/2.0, h/2.0
    cam[0,0] = cam[1,1] = 10.0
    return cv2.undistort(image, cam, distCoeff)

cv2.namedWindow("Distorsion", cv2.WINDOW_NORMAL)

# Crear las barras con los límites correctos
cv2.createTrackbar('K1', 'Distorsion', 500, 1000, nothing)  # rango 0–1000
cv2.createTrackbar('K2', 'Distorsion', 100, 200, nothing)   # rango 0–200

initial_values = {'K1': 500, 'K2': 100}
def reset_trackbars():
    for n, v in initial_values.items():
        cv2.setTrackbarPos(n, 'Distorsion', v)
    print("✅ Parámetros reiniciados")

while True:
    ret, frame = video.read()
    if not ret:
        video.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    k1 = (cv2.getTrackbarPos('K1', 'Distorsion') - 500) / 100000.0
    k2 = (cv2.getTrackbarPos('K2', 'Distorsion') - 100) / 1000000.0

    out = apply_distortion(frame, k1, k2)
    out = cv2.resize(out, (800, 400))

    tipo = "Barril" if k1 < 0 else "Almohadilla" if k1 > 0 else "Sin distorsión"
    texto = f"K1={k1:.5f}, K2={k2:.6f} | {tipo} | R=Reset, ESC=Salir"
    cv2.putText(out, texto, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    cv2.imshow("Distorsion", out)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break
    elif key in (ord('r'), ord('R')):
        reset_trackbars()

    # Cierre seguro (macOS fix)
    try:
        if cv2.getWindowProperty('Distorsion', cv2.WND_PROP_VISIBLE) < 1:
            break
    except cv2.error:
        break

video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


✅ Parámetros reiniciados


-1

#### Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en una imagen destino y realizar la transformación afín.

In [15]:
# Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en una imagen destino y realizar la transformación afín.

import cv2 as cv
import numpy as np

# ------------------------------
# Cargar imagen original
# ------------------------------
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # segunda ventana en blanco (imagen destino)

# Copias para dibujar los puntos sin modificar las imágenes base
clone1 = img.copy()
clone2 = img2.copy()

# Listas para guardar puntos de control en origen y destino
pts_src = []   # 3 puntos de la imagen original
pts_dst = []   # 3 puntos en la imagen destino

# ------------------------------
# Callback del ratón para ambas ventanas
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    """
    Registra los clics del usuario y dibuja los puntos seleccionados.
    El parámetro 'param' permite saber si el clic proviene de la ventana original (1)
    o de la ventana destino (2), porque OpenCV no identifica ventanas automáticamente.
    """
    global pts_src, pts_dst, clone1, clone2

    # Clic con el botón izquierdo del ratón
    if event == cv.EVENT_LBUTTONDOWN:

        # ----- Selección en la imagen original -----
        if param == 1 and len(pts_src) < 3:
            pts_src.append((x, y))  # guardar punto

            # Dibujar un círculo y numeración sobre la imagen clonada
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            cv.imshow('Original', clone1)

            # Cuando se eligen los 3 puntos del origen, avisamos
            if len(pts_src) == 3:
                print("3 puntos seleccionados en la imagen ORIGINAL")
                print("Ahora selecciona 3 puntos en la imagen DESTINO")

        # ----- Selección en la imagen destino -----
        elif param == 2 and len(pts_dst) < 3:
            pts_dst.append((x, y))

            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

            cv.imshow('Destino', clone2)

        # ----- Una vez se tienen 3 puntos en cada imagen -----
        if len(pts_src) == 3 and len(pts_dst) == 3:
            transformar_imagen()

# ------------------------------
# Función para realizar la transformación afín
# ------------------------------
def transformar_imagen():
    global pts_src, pts_dst, img, img2, clone1

    # La transformación afín necesita exactamente 3 pares de puntos.
    # A partir de ellos se construye la matriz 2x3 que aplica:
    # rotación + escala + traslación + cizalla (sin perspectiva).
    M = cv.getAffineTransform(np.float32(pts_src), np.float32(pts_dst))

    # cv.warpAffine aplica la matriz afín (2x3) a la imagen original.
    transformada = cv.warpAffine(img, M, (img2.shape[1], img2.shape[0]))

    # Mostrar la imagen original con puntos y la transformada
    cv.imshow('Imagen Original', clone1)
    cv.imshow('Imagen Transformada', transformada)


# ------------------------------
# Ventanas e interacción
# ------------------------------
cv.namedWindow('Original')
cv.namedWindow('Destino')

# Asignamos el callback de ratón a cada ventana indicando su "rol" con param
cv.setMouseCallback('Original', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("=" * 60)
print("TRANSFORMACIÓN AFÍN MEDIANTE 3 PUNTOS")
print("=" * 60)
print("1. Haz clic en 3 puntos en la imagen ORIGINAL (puntos rojos)")
print("2. Luego haz clic en 3 puntos en la imagen DESTINO (puntos azules)")
print("\nLos puntos deben estar en el MISMO ORDEN en ambas imágenes")
print("\nPresiona 'R' para reiniciar")
print("Presiona 'ESC' para salir")
print("=" * 60)

cv.imshow('Original', clone1)
cv.imshow('Destino', clone2)

# ------------------------------
# Loop principal
# ------------------------------
while True:
    key = cv.waitKey(1) & 0xFF
    
    if key == 27:  # ESC → salir
        break

    elif key == ord('r') or key == ord('R'):  # Reiniciar selección
        pts_src = []
        pts_dst = []
        clone1 = img.copy()
        clone2 = img2.copy()

        # Redibujar las ventanas sin puntos
        cv.imshow('Original', clone1)
        cv.imshow('Destino', clone2)

        # Cerrar ventanas de resultados si están abiertas (OpenCV no lo hace solo)
        cv.destroyWindow('Imagen Original')
        cv.destroyWindow('Imagen Transformada')

cv.destroyAllWindows()
cv.waitKey(1)


TRANSFORMACIÓN AFÍN MEDIANTE 3 PUNTOS
1. Haz clic en 3 puntos en la imagen ORIGINAL (puntos rojos)
2. Luego haz clic en 3 puntos en la imagen DESTINO (puntos azules)

Los puntos deben estar en el MISMO ORDEN en ambas imágenes

Presiona 'R' para reiniciar
Presiona 'ESC' para salir
3 puntos seleccionados en la imagen ORIGINAL
Ahora selecciona 3 puntos en la imagen DESTINO
3 puntos seleccionados en la imagen ORIGINAL
Ahora selecciona 3 puntos en la imagen DESTINO
3 puntos seleccionados en la imagen ORIGINAL
Ahora selecciona 3 puntos en la imagen DESTINO


-1

#### Calcular la imagen especular a partir de una imagen.

In [16]:
#### Calcular la imagen especular a partir de una imagen.
import cv2 as cv

# Cargar la imagen desde disco
img = cv.imread('images/eii.png')
if img is None:
    raise SystemExit("No se pudo cargar la imagen")

# Espejo horizontal (respecto al eje vertical): flipCode = 1
# flipCode =  1 → voltea horizontalmente (izquierda ↔ derecha)
# flipCode =  0 → voltea verticalmente   (arriba ↔ abajo)
# flipCode = -1 → voltea en ambos ejes
mirror_h = cv.flip(img, 1)

# Mostrar imagen original y su versión especular
cv.imshow('Original', img)
cv.imshow('Imagen especular', mirror_h)

# Esperar a que se pulse una tecla para cerrar las ventanas
cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)  # cierre robusto de ventanas


-1

#### Tratar una recta que será el eje de reflexión y “reflejar” la imagen.

In [17]:
#### Tratar una recta que será el eje de reflexión y “reflejar” la imagen.

import cv2 as cv
import numpy as np

img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]

# Variables globales que almacenan los dos puntos que definen la recta
punto1 = None
punto2 = None
puntos_completos = False

def mouse_callback(event, x, y, flags, param):
    """Captura dos puntos con clic izquierdo para definir el eje de reflexión."""
    global punto1, punto2, puntos_completos

    if event == cv.EVENT_LBUTTONDOWN:
        if punto1 is None:
            punto1 = (x, y)
            print(f"Primer punto: {punto1}")
        elif punto2 is None:
            punto2 = (x, y)
            puntos_completos = True
            print(f"Segundo punto: {punto2}")

def reflejar_imagen(img, p1, p2):
    """
    Refleja la imagen respecto a la línea definida por p1 y p2.
    Esta es la parte más matemática del ejercicio.
    """
    h, w = img.shape[:2]
    x1, y1 = p1
    x2, y2 = p2

    # Vector director de la recta
    dx = x2 - x1
    dy = y2 - y1

    # Longitud del vector (se usa para normalizar)
    length = np.sqrt(dx**2 + dy**2)
    if length == 0:
        # Si los dos puntos coinciden, no existe recta → no se puede reflejar
        return img, (0, 0), (w, h)

    # Normalización del vector director (dx, dy) → longitud 1
    dx /= length
    dy /= length

    # Fórmulas del reflejo respecto a una recta en R2:
    #   cos(2θ) = dx² - dy²
    #   sin(2θ) = 2·dx·dy
    #
    # Estas ecuaciones generan la matriz 2x2 que refleja cualquier punto
    cos_2theta = dx**2 - dy**2
    sin_2theta = 2 * dx * dy

    # Matriz de reflexión lineal (sin traslación)
    M_reflect = np.array([
        [cos_2theta, sin_2theta],
        [sin_2theta, -cos_2theta]
    ], dtype=np.float32)

    # PARA REFLEJAR RESPECTO A UNA RECTA NO PASADA POR ORIGEN:
    # Se traslada la recta al origen → se aplica reflexión → se deshace la traslación.
    translation = np.array([x1, y1], dtype=np.float32)

    # Offset que debe sumarse tras aplicar la matriz lineal
    # Fórmula: T' = p1 - M·p1
    reflected_translation = translation - M_reflect @ translation

    # Matriz completa 2x3 usada por warpAffine
    M = np.hstack([M_reflect, reflected_translation.reshape(2, 1)])

    # ------------------------------------------
    # AJUSTE DE TAMAÑO DEL LIENZO
    # ------------------------------------------
    # Se calcula dónde caerán las esquinas de la imagen reflejada.
    corners = np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.float32)

    # Transformación de cada esquina aplicando la reflexión pura
    corners_transformed = np.array([M_reflect @ (c - translation) + translation for c in corners])

    # Unimos esquinas originales y reflejadas para encontrar límites
    all_corners = np.vstack([corners, corners_transformed])

    # Buscar el rectángulo delimitador (bounding box)
    min_x, min_y = all_corners.min(axis=0)
    max_x, max_y = all_corners.max(axis=0)

    # Nuevo tamaño necesario del lienzo
    new_w = int(max(max_x - min_x, w * 2))
    new_h = int(max(max_y - min_y, h * 2))

    # Ajuste del término de traslación para que la imagen no quede fuera del lienzo
    M[0, 2] -= min_x
    M[1, 2] -= min_y

    # Aplicar transformación afín completa
    reflected = cv.warpAffine(img, M, (new_w, new_h))

    # Offset para recolocar gráficos superpuestos (como la recta)
    offset = (int(min_x), int(min_y))
    return reflected, offset, (new_w, new_h)

# ---------------------------
# Ventana principal
# ---------------------------
cv.namedWindow("Original")
cv.setMouseCallback("Original", mouse_callback)

print("Haz clic en dos puntos para definir el eje de reflexión")
print("Presiona 'R' para reiniciar los puntos")
print("Presiona 'ESC' para salir")

while True:
    img_display = img.copy()

    # Dibujar los puntos seleccionados
    if punto1 is not None:
        cv.circle(img_display, punto1, 5, (0, 255, 0), -1)
    if punto2 is not None:
        cv.circle(img_display, punto2, 5, (0, 255, 0), -1)

    # Dibujar la recta extendida que define el eje de reflexión
    if punto1 is not None and punto2 is not None:
        dx = punto2[0] - punto1[0]
        dy = punto2[1] - punto1[1]
        length = np.sqrt(dx**2 + dy**2)
        if length > 0:
            dx /= length
            dy /= length
            # Se extiende la recta más allá del segmento entre puntos
            extension = max(w, h) * 2
            p_start = (int(punto1[0] - dx * extension), int(punto1[1] - dy * extension))
            p_end = (int(punto1[0] + dx * extension), int(punto1[1] + dy * extension))
            cv.line(img_display, p_start, p_end, (255, 0, 255), 2)

    cv.putText(img_display, "Selecciona 2 puntos para el eje de reflexion",
               (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Mostrar imagen original con recta
    cv.imshow('Original', img_display)

    # Si ya tenemos la recta, mostrar también la imagen reflejada
    if puntos_completos:
        reflected, offset, size = reflejar_imagen(img, punto1, punto2)

        # Ajustar la posición de la recta en la imagen reflejada
        p1_adj = (punto1[0] - offset[0], punto1[1] - offset[1])
        p2_adj = (punto2[0] - offset[0], punto2[1] - offset[1])

        # Recalcular vector director
        dx = p2_adj[0] - p1_adj[0]
        dy = p2_adj[1] - p1_adj[1]
        length = np.sqrt(dx**2 + dy**2)

        if length > 0:
            dx /= length
            dy /= length
            extension = max(size)
            p_start = (int(p1_adj[0] - dx * extension), int(p1_adj[1] - dy * extension))
            p_end = (int(p1_adj[0] + dx * extension), int(p1_adj[1] + dy * extension))
            cv.line(reflected, p_start, p_end, (255, 0, 255), 2)

        cv.putText(reflected, "Imagen reflejada", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv.imshow("Reflejada", reflected)

    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break
    elif key in (ord('r'), ord('R')):
        # Reiniciar selección
        punto1 = None
        punto2 = None
        puntos_completos = False
        cv.destroyWindow("Reflejada")
        print("Puntos reiniciados")

cv.destroyAllWindows()
cv.waitKey(1)


Haz clic en dos puntos para definir el eje de reflexión
Presiona 'R' para reiniciar los puntos
Presiona 'ESC' para salir
Primer punto: (262, 47)
Segundo punto: (106, 256)
Puntos reiniciados


-1

#### Otras aportaciones: Zoom interactivo

In [18]:
import cv2 as cv
import numpy as np

# Variables globales
scale = 1.0
center_x, center_y = 0, 0
img_original = None
img_display = None
mouse_x, mouse_y = 0, 0

def aplicar_zoom(img, scale, center_x, center_y):
    """Aplica zoom a la imagen usando el centro especificado"""
    h, w = img.shape[:2]
    
    # Crear matriz de rotación con escala (rotación = 0)
    M = cv.getRotationMatrix2D((center_x, center_y), 0, scale)
    
    # Aplicar transformación
    img_zoom = cv.warpAffine(img, M, (w, h), flags=cv.INTER_LINEAR)
    
    return img_zoom

def mouse_callback(event, x, y, flags, param):
    """Callback para eventos del ratón"""
    global scale, center_x, center_y, img_original, img_display, mouse_x, mouse_y
    
    # Actualizar posición del ratón
    if event == cv.EVENT_MOUSEMOVE:
        mouse_x, mouse_y = x, y
    
    # Zoom con rueda del ratón (funciona en MacBook trackpad con scroll)
    if event == cv.EVENT_MOUSEWHEEL:
        # Actualizar centro del zoom a la posición actual del ratón
        center_x, center_y = x, y
        
        # En MacOS, flags contiene el delta del scroll
        # Positivo = scroll hacia arriba (zoom in)
        # Negativo = scroll hacia abajo (zoom out)
        if flags > 0:
            scale *= 1.15
        else:
            scale *= 0.85
        
        # Limitar escala entre 0.3 y 10.0
        scale = max(0.3, min(scale, 10.0))
        
        # Aplicar zoom
        img_display = aplicar_zoom(img_original, scale, center_x, center_y)
        actualizar_display()

def actualizar_display():
    """Actualiza la visualización con información"""
    global img_display, scale, center_x, center_y, mouse_x, mouse_y
    
    img_info = img_display.copy()
    
    # Información de zoom
    texto_zoom = f"Zoom: {scale:.2f}x"
    cv.putText(img_info, texto_zoom, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 
               0.8, (0, 255, 0), 2, cv.LINE_AA)
    
    # Información de centro
    texto_centro = f"Centro: ({center_x}, {center_y})"
    cv.putText(img_info, texto_centro, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 
               0.6, (0, 255, 0), 2, cv.LINE_AA)
    
    # Instrucciones
    texto_inst = "Trackpad: 2 dedos arriba/abajo | Teclado: +/- | R: Reset | ESC: Salir"
    cv.putText(img_info, texto_inst, (10, img_info.shape[0] - 15), 
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv.LINE_AA)
    
    # Dibujar cruz en el centro del zoom
    cv.drawMarker(img_info, (center_x, center_y), (0, 255, 0), 
                  cv.MARKER_CROSS, 25, 2)
    
    # Dibujar círculo en posición del ratón
    cv.circle(img_info, (mouse_x, mouse_y), 5, (255, 0, 255), -1)
    
    cv.imshow('Zoom Interactivo - MacBook', img_info)

# Cargar imagen de ejemplo
img_original = np.zeros((600, 800, 3), dtype=np.uint8)

# Crear un patrón de ejemplo más detallado
# Fondo degradado
for i in range(600):
    color = int(30 + (i / 600) * 50)
    cv.line(img_original, (0, i), (800, i), (color, color, color), 1)

# Grid más fino para ver el zoom en detalle
for i in range(0, 800, 25):
    color = (80, 80, 80) if i % 50 == 0 else (60, 60, 60)
    grosor = 2 if i % 50 == 0 else 1
    cv.line(img_original, (i, 0), (i, 600), color, grosor)
for i in range(0, 600, 25):
    color = (80, 80, 80) if i % 50 == 0 else (60, 60, 60)
    grosor = 2 if i % 50 == 0 else 1
    cv.line(img_original, (0, i), (800, i), color, grosor)

# Añadir círculos de colores con gradientes
colores = [(255, 50, 50), (50, 255, 50), (50, 50, 255), 
           (255, 255, 50), (255, 50, 255), (50, 255, 255)]
for i, color in enumerate(colores):
    x = 150 + (i % 3) * 250
    y = 150 + (i // 3) * 300
    # Círculo exterior
    cv.circle(img_original, (x, y), 90, color, -1)
    # Círculo medio
    cv.circle(img_original, (x, y), 60, tuple([c//2 for c in color]), -1)
    # Círculo interior
    cv.circle(img_original, (x, y), 30, (255, 255, 255), -1)
    # Número
    cv.putText(img_original, str(i+1), (x-15, y+15), 
               cv.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3, cv.LINE_AA)

# Añadir texto con sombra
texto = 'Zoom MacBook Trackpad'
cv.putText(img_original, texto, (152, 52), 
           cv.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 5, cv.LINE_AA)
cv.putText(img_original, texto, (150, 50), 
           cv.FONT_HERSHEY_SIMPLEX, 1.3, (255, 255, 255), 3, cv.LINE_AA)

# Si quieres cargar tu propia imagen:
# img_original = cv.imread('tu_imagen.jpg')

if img_original is None:
    print("Error: No se pudo cargar la imagen")
    exit()

# Inicializar centro en el medio de la imagen
h, w = img_original.shape[:2]
center_x, center_y = w // 2, h // 2
mouse_x, mouse_y = w // 2, h // 2

img_display = img_original.copy()

# Crear ventana y asignar callback
cv.namedWindow('Zoom Interactivo - MacBook')
cv.setMouseCallback('Zoom Interactivo - MacBook', mouse_callback)

# Mostrar instrucciones
print("=" * 60)
print("ZOOM INTERACTIVO")
print("=" * 60)
print("Instrucciones:")
print("  TRACKPAD:")
print("    - Desliza 2 dedos ARRIBA para ACERCAR (zoom in)")
print("    - Desliza 2 dedos ABAJO para ALEJAR (zoom out)")
print("    - El zoom se centra donde está el cursor")
print("")
print("  TECLADO (alternativa):")
print("    - Tecla '+' o '=' para acercar")
print("    - Tecla '-' para alejar")
print("    - Tecla 'R' para resetear el zoom")
print("    - Tecla 'ESC' o 'Q' para salir")
print("=" * 60)

# Bucle principal
while True:
    actualizar_display()
    
    # Esperar por tecla
    key = cv.waitKey(10) & 0xFF
    
    if key == 27 or key == ord('q'):  # ESC o Q para salir
        break
    elif key == ord('r'):  # R para resetear
        scale = 1.0
        center_x, center_y = w // 2, h // 2
        img_display = img_original.copy()
    elif key == ord('+') or key == ord('='):  # + para zoom in
        center_x, center_y = mouse_x, mouse_y
        scale *= 1.15
        scale = min(scale, 10.0)
        img_display = aplicar_zoom(img_original, scale, center_x, center_y)
    elif key == ord('-'):  # - para zoom out
        center_x, center_y = mouse_x, mouse_y
        scale *= 0.85
        scale = max(scale, 0.3)
        img_display = aplicar_zoom(img_original, scale, center_x, center_y)

cv.destroyAllWindows()
print("\n¡Programa finalizado.")
cv.waitKey(1)  # cierre robusto de ventanas


ZOOM INTERACTIVO
Instrucciones:
  TRACKPAD:
    - Desliza 2 dedos ARRIBA para ACERCAR (zoom in)
    - Desliza 2 dedos ABAJO para ALEJAR (zoom out)
    - El zoom se centra donde está el cursor

  TECLADO (alternativa):
    - Tecla '+' o '=' para acercar
    - Tecla '-' para alejar
    - Tecla 'R' para resetear el zoom
    - Tecla 'ESC' o 'Q' para salir

¡Programa finalizado.


-1